# Publish trained model

In [ ]:
!pip install boto3
!pip install gitpython
!pip install pyyaml

from datetime import datetime
import os

import boto3
import git
import yaml

from deployment import get_deployment_resource

Read and initialize parameters

In [ ]:
s3_endpoint_url = os.environ.get('S3_ENDPOINT_URL')
s3_access_key = os.environ.get('S3_ACCESS_KEY')
s3_secret_key = os.environ.get('S3_SECRET_KEY')
s3_bucket_prefix = 'anomaly-detection'

timestamp = datetime.now().strftime('%y%m%d%H%M')
s3_bucket_name = f'{s3_bucket_prefix}-{timestamp}'

ops_repo_location = os.environ.get('OPS_REPO_LOCATION')
git_user = os.environ.get('GIT_USER')
git_password = os.environ.get('GIT_PASSWORD')
ops_repo_url = (
    f'http://{git_user}:{git_password}@{ops_repo_location.lstrip("https://")}'
)

Upload model to S3 bucket

In [ ]:
print(f'Uploading model to bucket {s3_bucket_name}'
      f'to S3 storage at {s3_endpoint_url}')
s3_client = boto3.client(
    's3', endpoint_url=s3_endpoint_url,
    aws_access_key_id=s3_access_key, aws_secret_access_key=s3_secret_key
)
s3_client.create_bucket(Bucket=s3_bucket_name)
with open('model.joblib', 'rb') as model_file:
    s3_client.upload_fileobj(model_file, s3_bucket_name, 'model.joblib')

## Push changes to Git

Clone ops repository

In [ ]:
print(f'Checking out repo at {ops_repo_location} with user {git_user}')
ops_repository_local = '/opt/app-root/src/industrial-edge'
try:
    repository = git.Repo.clone_from(ops_repo_url, ops_repository_local)
except git.GitCommandError as error:
    print(f'Git clone failed: {error}\nChecking out local repository.')
    repository = git.Repo(ops_repository_local)

with repository.config_writer() as git_config:
    git_config.set_value('user', 'name', git_user)

Update Seldon Deployment CR in test environment

In [ ]:
seldon_deployment_file_name_test = (
    f'{ops_repository_local}/charts/datacenter/manuela-tst/templates/'
    f'anomaly-detection/iot-anomaly-detection-seldon.yaml'
)

seldon_deployment_cr = get_deployment_resource(s3_bucket_name)

print(f'Writing updated Seldon Deployment CR: {seldon_deployment_cr}')

with open(seldon_deployment_file_name_test, 'w') as outputfile:
    yaml.safe_dump(seldon_deployment_cr, outputfile)

repository.index.add(seldon_deployment_file_name_test)
repository.index.commit(f'Model update {timestamp} in test environment.')
repository.remotes.origin.push()

Prepare Seldon Deployment CR update in production

In [ ]:
branch_name = f'model-update-{timestamp}'
repository.git.branch(branch_name)
repository.git.checkout(branch_name)

seldon_deployment_file_name_prod = (
    f'{ops_repository_local}/charts/factory/manuela-stormshift/templates/'
    f'anomaly-detection/iot-anomaly-detection-seldon.yaml'
)
with open(seldon_deployment_file_name_prod, 'r') as inputfile:
    seldon_deployment_cr = yaml.safe_load(inputfile)

print(f'Read Seldon Deployment CR: {seldon_deployment_cr}')

new_model_uri = f's3://{s3_bucket_name}'
seldon_deployment_cr['spec']['predictors'][0]['graph']['modelUri'] = new_model_uri

print(f'Writing updated Seldon Deployment CR: {seldon_deployment_cr}')

with open(seldon_deployment_file_name_prod, 'w') as outputfile:
    yaml.safe_dump(seldon_deployment_cr, outputfile)

repository.index.add(seldon_deployment_file_name_prod)
repository.index.commit(f'Model update {timestamp} in production.')
repository.git.push('--set-upstream', 'origin', branch_name)